Shishir Poreddy - Jimmy John's - ECON433

Part 1


In [3]:
#1a
import pandas as pd
filename = [
    '/Users/shishirporeddy/ECON433/weekly_patterns_2018_sample.csv.gz',
    '/Users/shishirporeddy/ECON433/weekly_patterns_2019_sample.csv.gz',
    '/Users/shishirporeddy/ECON433/weekly_patterns_2020_sample.csv.gz',
    '/Users/shishirporeddy/ECON433/weekly_patterns_2021_sample.csv.gz',
    '/Users/shishirporeddy/ECON433/weekly_patterns_2022_sample.csv.gz'
]
row_dictionary = {}
jj_list = []

for file in filename:
    datax = pd.read_csv(file, compression='gzip', low_memory= False)
    row_dictionary[file] = len(datax)
    print(datax.columns)
    jj_data = datax[datax['brands'] =="Jimmy John's"]
    jj_list.append(jj_data)
count = 0
    
for file, row in row_dictionary.items():
    print(f"{file}: {count} rows")
    
row_tot = sum(row_dictionary.values())
print(row_tot)

concat_jj_data = pd.concat(jj_list, ignore_index=True)
print(concat_jj_data.head())

    


Index(['placekey', 'safegraph_brand_ids', 'brands', 'top_category',
       'naics_code', 'city', 'region', 'postal_code', 'date_range_start',
       'date_range_end', 'raw_visit_counts', 'visits_by_day'],
      dtype='object')
Index(['placekey', 'safegraph_brand_ids', 'brands', 'top_category',
       'naics_code', 'city', 'region', 'postal_code', 'date_range_start',
       'date_range_end', 'raw_visit_counts', 'visits_by_day'],
      dtype='object')
Index(['placekey', 'safegraph_brand_ids', 'brands', 'top_category',
       'naics_code', 'city', 'region', 'postal_code', 'date_range_start',
       'date_range_end', 'raw_visit_counts', 'visits_by_day'],
      dtype='object')
Index(['placekey', 'safegraph_brand_ids', 'brands', 'top_category',
       'naics_code', 'city', 'region', 'postal_code', 'date_range_start',
       'date_range_end', 'raw_visit_counts', 'visits_by_day'],
      dtype='object')
Index(['placekey', 'safegraph_brand_ids', 'brands', 'top_category',
       'naics_code', 'ci

In [4]:
#1b & 1c
print(len(concat_jj_data))
print(concat_jj_data.dtypes)

jj_unique = concat_jj_data['placekey'].nunique()
print(jj_unique)

concat_jj_data.to_csv('/Users/shishirporeddy/ECON433/jimmy_johns_wide_sample.csv', index = False)


104400
placekey               object
safegraph_brand_ids    object
brands                 object
top_category           object
naics_code             object
city                   object
region                 object
postal_code            object
date_range_start       object
date_range_end         object
raw_visit_counts       object
visits_by_day          object
dtype: object
400


Question 1a
- There are a total of 1055229 observations in each individual dataset and a total of 5196505 observations among all the datasets.
- The unit of observation in my dataset is a specific restauarant represented by 'placekey' during a designated start and end date

Question 1b:
- There are now a total of 104400 observations

Question 1c: concat_jj_data = "wide sample"
- The data type is an "object" type in every variable in my wide sample
- There a total of 400 unique Jimmy Johh's locations in the US
- Online shows that there are roughly 2600 Jimmy John's locations, which could mean that my data might be incomplete and hasn't gathered data from all the states. 

Part 2

In [5]:
#2a
import datetime
concat_jj_data['date_range_start'] = pd.to_datetime(concat_jj_data['date_range_start'], utc=True).dt.tz_localize(None)
concat_jj_data['date_range_end'] = pd.to_datetime(concat_jj_data['date_range_end'], utc=True).dt.tz_localize(None)

print(concat_jj_data.dtypes)

start_date= concat_jj_data['date_range_start'].min()
end_date= concat_jj_data['date_range_end'].max()

print(start_date)
print(end_date)

placekey                       object
safegraph_brand_ids            object
brands                         object
top_category                   object
naics_code                     object
city                           object
region                         object
postal_code                    object
date_range_start       datetime64[ns]
date_range_end         datetime64[ns]
raw_visit_counts               object
visits_by_day                  object
dtype: object
2018-01-01 05:00:00
2023-01-02 08:00:00


In [6]:
#2b
concat_jj_data[['dailyvisits1','dailyvisits2', 'dailyvisits3', 'dailyvisits4', 'dailyvisits5', 'dailyvisits6', 'dailyvisits7']] = concat_jj_data['visits_by_day'].str.strip("[]").str.split(",", expand=True).apply(pd.to_numeric, errors='coerce')
print(len(concat_jj_data.columns))

19


In [24]:
#2c
jj_long = concat_jj_data.melt(
    id_vars=['placekey', 'date_range_start', 'date_range_end', 'region', 'brands', 'city', 'raw_visit_counts', 'visits_by_day'],
    value_vars=['dailyvisits1','dailyvisits2', 'dailyvisits3', 'dailyvisits4', 'dailyvisits5', 'dailyvisits6', 'dailyvisits7'],
    var_name='visits_column',
    value_name='dailyvisits')

jj_long['dailyvisits'] = pd.to_numeric(jj_long['dailyvisits'], errors='coerce').fillna(0).astype(int)

print(len(jj_long))

jj_long.to_csv("/Users/shishirporeddy/ECON433/jimmy_johns_long_sample.csv", index=False)


730800


In [14]:
#2d
dailyvisits_sum_stats = jj_long['dailyvisits'].describe()
print(dailyvisits_sum_stats)

Q_one = jj_long['dailyvisits'].quantile(0.25)
Q_three = jj_long['dailyvisits'].quantile(0.75)
inter_quart_range = Q_three - Q_one

low_outlier_range = Q_one - 1.5 * inter_quart_range
upp_outlier_range = Q_three + 1.5 * inter_quart_range

outliers = jj_long[~jj_long['dailyvisits'].between(low_outlier_range, upp_outlier_range)]      
print(len(outliers))

missing_values = jj_long['dailyvisits'].isna().sum()
print(missing_values)



count    730800.000000
mean         15.393637
std         156.931828
min           0.000000
25%           1.000000
50%           3.000000
75%           7.000000
max        5915.000000
Name: dailyvisits, dtype: float64
49776
0


Question 2a:
The start date is January 1, 2018 at 5:00 am and the end date is January 2, 2023 at 8:00am for my wide sample.

Question 2b:
There are now 19 variables in the wide sample now. 

Question 2c:
There are now 730,800 observations in the long sample. 
The pros for the long sample is that it is much easier to analyze the data by day instead of by week. The cons of the long sample are the extra storage and power used since there are more rows. 
After converting dailyvisits directly into an integer the code doesn't work because you can't convert (NA or inf) to integer, so I added .nafill(0) to resolve this issue. 
There are still 730,800 observations. 

Question 2d:
Mean = 15.39, STD = 156.93, minimum = 0, maximum = 5915
Using interquartile range, I found there to be 49,776 outliers. 
There are no missing values in dailyvisits. 
The economic reason for these missing values could simply be due to holiday closures. The data could tell us this by seeing when exactly the misssing values occured and if they land during holiday seasons. 
Outliers and missing data can result in the business owner to read a different story than what the data is actually saying. One example of a sensitive issue would be in restauarants and trying to schedule the staff on a day by day basis. One example of an immune issue would be something longterm, such as marketing campaings.  

Part 3

In [15]:
#3a:
jj_long['date_range_start'] = pd.to_datetime(jj_long['date_range_start'], errors = 'coerce')
jj_long['year'] = jj_long['date_range_start'].dt.year

sum_of_yr = (jj_long.groupby('year', as_index=False).agg(start_date=('date_range_start', 'min'), end_date=('date_range_end', 'max'), observations=('dailyvisits', 'count'), mean_visits=('dailyvisits', 'mean')))

print(sum_of_yr)
            


   year          start_date            end_date  observations  mean_visits
0  2018 2018-01-01 05:00:00 2019-01-07 08:00:00        148400     8.461044
1  2019 2019-01-07 05:00:00 2020-01-06 08:00:00        145600    23.857225
2  2020 2020-01-06 05:00:00 2021-01-04 08:00:00        145600    11.465797
3  2021 2021-01-04 05:00:00 2022-01-03 08:00:00        145600    14.197562
4  2022 2022-01-03 05:00:00 2023-01-02 08:00:00        145600    19.119876


In [16]:
#3b
jj_long['dayofweek'] = jj_long['date_range_start'].dt.day_name()

print(jj_long[[ 'date_range_start', 'dayofweek']].head())

unique_days = jj_long['dayofweek'].nunique()
print(unique_days)

summary_dayofweek=(jj_long.groupby('dayofweek', as_index=False).agg(observations=('dailyvisits', 'count'), mean_dailyvisits=('dailyvisits', 'mean')))
print(summary_dayofweek)

highest_dv = summary_dayofweek.loc[summary_dayofweek['mean_dailyvisits'].idxmax(), 'dayofweek']
print(highest_dv)

     date_range_start dayofweek
0 2018-01-01 06:00:00    Monday
1 2018-01-01 06:00:00    Monday
2 2018-01-01 05:00:00    Monday
3 2018-01-01 05:00:00    Monday
4 2018-01-01 08:00:00    Monday
1
  dayofweek  observations  mean_dailyvisits
0    Monday        730800         15.393637
Monday


In [17]:
#3c 
import numpy as np

jj_long['date_range_start'] = pd.to_datetime(jj_long['date_range_start'], errors = 'coerce')
jj_long['day_name'] = jj_long['date_range_start'].dt.day_name()
jj_long['WKND'] = jj_long['day_name'].isin(['Saturday', 'Sunday'])
jj_long['threshold'] = np.where(jj_long['WKND'], 13, 18)
jj_long['manyvisits'] = np.where(jj_long['dailyvisits'] > jj_long['threshold'], 1, 0)
print(jj_long[['date_range_start', 'day_name', 'WKND', 'threshold', 'dailyvisits', 'manyvisits']].head(10))

geographic_var = 'city'
geo_summary = (
    jj_long.groupby(geographic_var, as_index=False).agg(tot_obs=('manyvisits', 'count'),count_manyvisits=('manyvisits', 'sum')).assign(percentage_manyvisits=lambda df:df['count_manyvisits']/df['tot_obs']))

max_count=geo_summary['count_manyvisits'].idxmax()
max_percentage=geo_summary['percentage_manyvisits'].idxmax()

max_count_region = geo_summary.loc[max_count, geographic_var]
max_count_value = geo_summary.loc[max_count, 'count_manyvisits']
max_percentage_region = geo_summary.loc[max_percentage, geographic_var]
max_percentage_value = geo_summary.loc[max_percentage, 'percentage_manyvisits']

print(geographic_var)
print(geo_summary.head(10))

print(f"Geographic unit with the highest # is: {max_count_region} with {max_count_value}")
print(f"Geographic unit with the highest % is: {max_percentage_region} at {max_percentage_value}")


     date_range_start day_name   WKND  threshold  dailyvisits  manyvisits
0 2018-01-01 06:00:00   Monday  False         18            0           0
1 2018-01-01 06:00:00   Monday  False         18            0           0
2 2018-01-01 05:00:00   Monday  False         18            0           0
3 2018-01-01 05:00:00   Monday  False         18            0           0
4 2018-01-01 08:00:00   Monday  False         18            3           0
5 2018-01-01 08:00:00   Monday  False         18            2           0
6 2018-01-01 06:00:00   Monday  False         18            1           0
7 2018-01-01 08:00:00   Monday  False         18            0           0
8 2018-01-01 06:00:00   Monday  False         18            0           0
9 2018-01-01 06:00:00   Monday  False         18            0           0
city
             city  tot_obs  count_manyvisits  percentage_manyvisits
0         Abilene     1827                 2               0.001095
1         Addison     1827                 0 

In [20]:
#3d

core_qualification = 15


 
unique_in_region = (
    jj_long.groupby('region', as_index=False)['placekey'].nunique().rename(columns={'placekey':'number_of_stores'}))

jj_long = jj_long.merge(unique_in_region, on = 'region', how='left')

jj_long['core_biz_area'] = np.where(jj_long['number_of_stores'] >= core_qualification, 1, 0)

core_sum =(
    jj_long.groupby('core_biz_area', as_index = False).agg(observations = ('dailyvisits', 'count'), mean_dailyvisits=('dailyvisits', 'mean')))
print(core_sum)

jj_long.to_csv("jimmy_johns_long_sample.csv", index=False)


   core_biz_area  observations  mean_dailyvisits
0              0        314244         22.316194
1              1        416556         10.171358


Question 3a:
Generated table summarizing start and end dates, # of observations in each year, and mean dailyvisits per year

Question 3b:
The day of week with the highest dailyvisits in my long sample is Monday.

Question 3c:
I chose this threshold because monday was the highest mean per day of week, so I set a threshold of 18 because it seems weekdays are busier than weekends. 
Chicago is the highest number of observations with manyvisits = 1. 
Little River has the highest percentage of observations with manyvisits = 1. 
This data makes sense in real life because Illinois has the most number of locations in the U.S.

Question 3d:
The number of observations for non-core areas is 314, 244 and for core areas it is 416,556. The mean daily visits for non-core areas is 22.32 and for core areas it is 10.17. 
On average I observe more daily visits in non-core areas. 
One reason for this is because of competition in the area, in non-core areas the jimmy john's might be the only sandwhich shop in the area meaning anytime someone wants a sandwhich they'll go to Jimmy John's where as in the core areas they might be other options like subway, potbelly, or jersey mikes which could lead them to have a lower daily average. Another reason might be due to pricing differences, in core areas the price of sandwhiches at Jimmy Johns might be more expensive, while at non-core areas the price could be cheaper. This could lead more people to enjoy jimmy johns in non-core areas because of its lower price point. 

In [32]:
import pandas as pd

# Suppose you melted into a column called 'visits_column' with values like 'dailyvisits1'..'dailyvisits7'
# 1) Extract the day index (1..7)
jj_long["day_index"] = (
    jj_long["visits_column"].str.extract(r'(\d+)$')  # capture the digit at the end
                   .astype(int)
)

# 2) Create a true daily date by adding (day_index - 1) days to the start date
jj_long["true_date"] = jj_long["date_range_start"] + pd.to_timedelta(jj_long["day_index"] - 1, unit="D")

# 3) Compute the actual weekday name
jj_long["day_of_week"] = jj_long["true_date"].dt.day_name()

# Now you'll see Monday for day_index=1, Tuesday for day_index=2, etc.
unique_days = jj_long["day_of_week"].unique()
print("Unique values in day_of_week:", unique_days)

# 2) Generate weekend variable
jj_long['WKND'] = jj_long['day_of_week'].isin(['Saturday', 'Sunday']).astype(int)

# 3) Count weekend vs. weekday observations
weekend_count = (jj_long['WKND'] == 1).sum()
weekday_count = (jj_long['WKND'] == 0).sum()

print("Weekend observations:", weekend_count)
print("Weekday observations:", weekday_count)


Unique values in day_of_week: ['Monday' 'Tuesday' 'Wednesday' 'Thursday' 'Friday' 'Saturday' 'Sunday']
Weekend observations: 208800
Weekday observations: 522000
